In [1]:
#Revisión de que spacy este instalado
import spacy
print(spacy.__version__)


3.7.2


In [17]:
#Llamamos a las librerias que vamos a necesitar
import spacy
import pathlib
import pandas as pd
from collections import Counter
from spacy import displacy
from spacy.matcher import Matcher

